# Proyecto 1: Etapa 2 - Inteligencia de negocios

**Nicolas Camargo - 202020782**

**Pablo Pedreros - 202112491**

**Jairo Garavito Correa - 202111499**

In [1]:
# Instalación de librerias
import pandas as pd
import numpy as np
import sys
import statistics
import nltk

import re, string, unicodedata
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('../../tipo1_entrenamiento_estudiantes.csv', delimiter=',')

dfData = df
dfData

,Review,Class
0,Nos alojamos en una casa alquilada en la ciuda...,4
1,"La comida está bien, pero nada especial. Yo te...",3
2,"En mi opinión, no es una como muchos usuarios ...",3
3,esta curiosa forma que asemeja una silla de mo...,4
4,Lo mejor era la limonada. Me gusto la comida d...,2
...,...,...
7870,El motivo de mi estancia fue porque vine a un ...,3
7871,Es difícil revisar el castillo porque apenas p...,3
7872,Si vas a Mérida no puedes perderte de este lug...,5
7873,"Este imperdible sitio, que lleva el nombre del...",5


## Limpieza de datos

In [8]:
dfData.drop_duplicates(inplace=True)

In [4]:
import preprocessing as pp
from sklearn.preprocessing import FunctionTransformer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['Hola', ',', 'como', 'estas', '?', 'Estoy', 'bien', ',', 'gracias', '.'], ['Seugnda', 'fila', 'con', 'texto']]
[['hola', 'bien', 'gracias'], ['seugnda', 'fila', 'texto']]
[['hol', 'bien', 'graci', 'hola', 'bien', 'gracias'], ['seugnd', 'fil', 'text', 'seugnda', 'fila', 'texto']]
['hol bien graci hola bien gracias', 'seugnd fil text seugnda fila texto']
(2, 11)
TF-IDF del primer documento:
[0.70710678 0.         0.         0.35355339 0.35355339 0.35355339
 0.35355339 0.         0.         0.         0.        ]


## Tokenización

In [5]:
dfData['words'] = dfData['Review'].apply(word_tokenize)
dfData.head()

,Review,Class,words
0,Nos alojamos en una casa alquilada en la ciuda...,4,"[Nos, alojamos, en, una, casa, alquilada, en, ..."
1,"La comida está bien, pero nada especial. Yo te...",3,"[La, comida, está, bien, ,, pero, nada, especi..."
2,"En mi opinión, no es una como muchos usuarios ...",3,"[En, mi, opinión, ,, no, es, una, como, muchos..."
3,esta curiosa forma que asemeja una silla de mo...,4,"[esta, curiosa, forma, que, asemeja, una, sill..."
4,Lo mejor era la limonada. Me gusto la comida d...,2,"[Lo, mejor, era, la, limonada, ., Me, gusto, l..."


In [6]:
dfData['Tokens'] = dfData['words'].apply(pp.preprocessingTransformation)
dfData.head()

KeyboardInterrupt: 

## Normalización

In [ ]:
dfData['Tokens'] = dfData['Tokens'].apply(pp.stem_and_lemmatize)
dfData.head()

,Review,Class,words,Tokens
0,Nos alojamos en una casa alquilada en la ciuda...,4,"[Nos, alojamos, en, una, casa, alquilada, en, ...","[aloj, cas, alquil, ciud, amurall, pareci, tan..."
1,"La comida está bien, pero nada especial. Yo te...",3,"[La, comida, está, bien, ,, pero, nada, especi...","[com, bien, especial, teni, mejor, com, mexc, ..."
2,"En mi opinión, no es una como muchos usuarios ...",3,"[En, mi, opinión, ,, no, es, una, como, muchos...","[opinion, usuari, reclam, gran, palad, parec, ..."
3,esta curiosa forma que asemeja una silla de mo...,4,"[esta, curiosa, forma, que, asemeja, una, sill...","[curios, form, asemej, sill, mont, ahi, nombr,..."
4,Lo mejor era la limonada. Me gusto la comida d...,2,"[Lo, mejor, era, la, limonada, ., Me, gusto, l...","[mejor, limon, gust, com, mund, sos, fri, mejo..."


## Selección de campos

In [ ]:
dfData['Tokens'] = dfData['Tokens'].apply(lambda x: ' '.join(map(str, x)))
dfData

,Review,Class,words,Tokens
0,Nos alojamos en una casa alquilada en la ciuda...,4,"[Nos, alojamos, en, una, casa, alquilada, en, ...",aloj cas alquil ciud amurall pareci tan segur ...
1,"La comida está bien, pero nada especial. Yo te...",3,"[La, comida, está, bien, ,, pero, nada, especi...",com bien especial teni mejor com mexc unid mar...
2,"En mi opinión, no es una como muchos usuarios ...",3,"[En, mi, opinión, ,, no, es, una, como, muchos...",opinion usuari reclam gran palad parec ser par...
3,esta curiosa forma que asemeja una silla de mo...,4,"[esta, curiosa, forma, que, asemeja, una, sill...",curios form asemej sill mont ahi nombr icon ci...
4,Lo mejor era la limonada. Me gusto la comida d...,2,"[Lo, mejor, era, la, limonada, ., Me, gusto, l...",mejor limon gust com mund sos fri mejor limona...
...,...,...,...,...
7870,El motivo de mi estancia fue porque vine a un ...,3,"[El, motivo, de, mi, estancia, fue, porque, vi...",motiv estanci vin congres medic hosped lug ins...
7871,Es difícil revisar el castillo porque apenas p...,3,"[Es, difícil, revisar, el, castillo, porque, a...",dificil revis castill apen podi camin sofoc ca...
7872,Si vas a Mérida no puedes perderte de este lug...,5,"[Si, vas, a, Mérida, no, puedes, perderte, de,...",si vas mer pued perdert lug nuev sucursal mas ...
7873,"Este imperdible sitio, que lleva el nombre del...",5,"[Este, imperdible, sitio, ,, que, lleva, el, n...",imperd siti llev nombr conquist joy urbanasu a...


In [ ]:
X_data, y_data = dfData['Tokens'],dfData['Class']

## Vectorización

In [10]:
tfidf_vectorizer = TfidfVectorizer()

# Ajustar y transformar los documentos
X_tf_idf = tfidf_vectorizer.fit_transform(X_data)
print(X_tf_idf.shape)
# Mostrar la representación TF-IDF del primer documento
print("TF-IDF del primer documento:")
print(X_tf_idf.toarray()[0])

NameError: name 'X_data' is not defined

## Pipeline con Support Vector Machine

In [11]:
from joblib import dump, load
pipeline = Pipeline(
    [
        ("tokenization", pp.tokenization_trans),
        ("preprocessing", pp.preprocessing_trans),
        ("stem_and_lemm", pp.stem_and_lemm_trans),
        ("join", pp.join_trans),
        ("vectorizer", tfidf_vectorizer),
        ("SVC", SVC(C=10))
    ]
)

In [12]:
X = dfData['Review']
Y = dfData['Class']

In [13]:
# Para entrenar el pipeline
X = dfData['Review']
Y = dfData['Class']
pipeline.fit(X, Y)

Pipeline(steps=[('tokenization',
                 FunctionTransformer(func=<function tokenize_words at 0x00000170AC9E1B40>)),
                ('preprocessing',
                 FunctionTransformer(func=<function preprocessingTransformation at 0x00000170AC9E1AB0>)),
                ('stem_and_lemm',
                 FunctionTransformer(func=<function stem_and_lemmatize at 0x00000170AC9E1BD0>)),
                ('join',
                 FunctionTransformer(func=<function juntar at 0x00000170AC9E1C60>)),
                ('vectorizer', TfidfVectorizer()), ('SVC', SVC(C=10))])

In [14]:
dump(pipeline, 'modelo.joblib')

['modelo.joblib']